### Importing Libraries:

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import adfuller
import requests

### data collection and stationarity analysis plan 

In [ ]:
# For debugging:
VERBOSE = False


# Fetch top-ranked cryptocurrencies by market cap (extra fetched to handle missing Yahoo data)

API_KEY = "YOUR_API_KEY"     #Api Key for coinmarketcap website

url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": API_KEY
}

params = {
    "start": "1",
    "limit": "70",
    "convert": "USD"
}

response = requests.get(url, headers=headers, params=params)
data = response.json()["data"]
top70_df = pd.DataFrame([
    {
        # "id": coin["id"],
        # "name": coin["name"],
        "symbol": coin["symbol"],
        # "market_cap": coin["quote"]["USD"]["market_cap"]
    }
    for coin in data
])



if VERBOSE:
    print("The Best 50 cryptocurrencies based on market cap:")
    print(top70_df)
    print("-------")




excluded_cryptos =[]  # Excluded cryptocurrencies due to missing Yahoo Finance data
constant_cryptos=[]   # adftest is not applicable for these cryptocurrencies
adf_results={}        # # Dictionary containing ADF test statistics for applicable cryptocurrencies


def perform_adf_test(crypto_symbol, serie):
    clean_serie = serie.dropna()

    # Skip ADF test if there is insufficient price data
    if len(clean_serie) < 30:
        if VERBOSE:
            print(f"{crypto_symbol}: ADF skipped (insufficient data length)")
        return

    # Skip ADF test if the price series is constant or nearly constant
    if clean_serie.nunique() <= 1:
        constant_cryptos.append(crypto_symbol)
        if VERBOSE:
            print(f"{crypto_symbol}: ADF skipped (constant or insufficient variation)")
        return

    result = adfuller(clean_serie)

    ADF_Statistic = result[0]
    p_value = result[1]
    critical_values = result[4]

    
    # adf_results stores cryptocurrencies identified as stationary at the 5% significance level (p-value < 0.05)
    if p_value < 0.05:
        adf_results[crypto_symbol] = {
            "ADF_Statistic": ADF_Statistic,
            "p_value": p_value,
            "critical_value_5%": critical_values["5%"]
        }

    if VERBOSE:
        print(f"important statistics of {crypto_symbol}:")
        print(f"ADF-Statistic: {ADF_Statistic}")
        print(f"p-value: {p_value}")
        print(f"critical-values: {critical_values}")
        print("------")




# this is a dataframe of 50 columns and each column name is the symbol of a cryptocurrency 
# and the rows of each column is the close price of corresponding cryptocurrency from '2024-12-01' to '2025-12-01' in one day timeframe.
pricedata_currencies_df =pd.DataFrame([])

counter= 0        # this is used to exactly get 50 cryptocurrency.if one is not in the yahoo then this will not be incremented vecause we want to have exactly 50 currency with its price data.
df_rowpointer= 0   # this is a pointer to the rows of the top70_df.it is used to get the value of pointed row and the column named 'symbol'
while(counter<51 and df_rowpointer<len(top70_df)):  #second condition is just added for rebustness by preventing access beyond the
                                                    # available ranked cryptocurrencies if sufficient valid price data is not found
        symbolname=top70_df.iloc[df_rowpointer]['symbol']
        df_rowpointer+=1
        tickerSymbol = symbolname+"-USD"
        tickerData = yf.Ticker(tickerSymbol)
        tickerDf = tickerData.history(interval='1d', start='2024-12-01', end='2025-12-01')
        if tickerDf.empty or tickerDf['Close'].isna().all():
            excluded_cryptos.append(tickerSymbol)
            continue
        counter+=1
        priceData = tickerDf.Close
        pricedata_currencies_df[tickerSymbol] = priceData



if VERBOSE:
    print("The Exluded cryptocurrencies because on no data in yahoo:")
    for x in excluded_cryptos:
        print(x)

    print("-------")

    print("the count of currencies (from those 50 currencies) that their data is available in yahoo finance:")
    print(len(pricedata_currencies_df.columns))

    print("------")




# plt.figure(figsize=(20,10))
# plt.plot(priceData)
# plt.title("%s Price Data"%tickerSymbol, fontsize=20)



for crypto_symbol in pricedata_currencies_df.columns: 
    perform_adf_test(crypto_symbol,pricedata_currencies_df[crypto_symbol])




for symbol, res in adf_results.items():
    print(f"{symbol}, {res['p_value']:.4f}")

print(len(adf_results))


$HYPE-USD: possibly delisted; no price data found  (1d 2024-12-01 -> 2025-12-01)
$SUI-USD: possibly delisted; no price data found  (1d 2024-12-01 -> 2025-12-01)
$MNT-USD: possibly delisted; no price data found  (1d 2024-12-01 -> 2025-12-01)
$CC-USD: possibly delisted; no price data found  (1d 2024-12-01 -> 2025-12-01)
$TAO-USD: possibly delisted; no price data found  (1d 2024-12-01 -> 2025-12-01)
$PEPE-USD: possibly delisted; no price data found  (1d 2024-12-01 -> 2025-12-01)
$POL-USD: possibly delisted; no price data found  (1d 2024-12-01 -> 2025-12-01)


USDT-USD, 0.0010
XRP-USD, 0.0415
USDC-USD, 0.0070
ZEC-USD, 0.0036
XLM-USD, 0.0231
USDe-USD, 0.0012
LTC-USD, 0.0389
DAI-USD, 0.0000
SHIB-USD, 0.0296
PYUSD-USD, 0.0000
DOT-USD, 0.0001
USD1-USD, 0.0059
BGB-USD, 0.0230
NEAR-USD, 0.0139
PI-USD, 0.0048
ICP-USD, 0.0372
RLUSD-USD, 0.0004
WLD-USD, 0.0140
ARB-USD, 0.0472
NIGHT-USD, 0.0058
20
